In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import sys
#change to any directory you have to store the repo
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling')

from src.features.metric import diversity, get_topic_coherence
from models.atten_model import MODEL_ATT_COMP
import swifter
from src.models.utils import sinkhorn_torch
from sklearn.preprocessing import MultiLabelBinarizer


In [48]:
import pandas as pd
from collections import Counter
from stop_words import get_stop_words
import swifter
from scipy.sparse import csr_matrix, save_npz
import numpy as np
import gc
import torch
stop_words = get_stop_words('en')
import re
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk.corpus import wordnet
import torch
from torch.nn.functional import normalize
import pickle 
import torch.nn.functional as F 
import numpy as np 
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.nn import init
from models.NVDM import topic_covariance_penalty, sinkhorn_torch, NTM, negative_sampling_prior, optimal_transport_prior,  NormalParameter, get_mlp, EmbTopic, NSSTM, OTETM
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words, generate_bow
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize

In [ ]:
#data import
news = pd.read_csv('https://raw.githubusercontent.com/yumeng5/WeSTClass/master/agnews/dataset.csv', 
                   error_bad_lines=False,
                   names = ['class', 'title', 'description'])
news['text'] = news.swifter.apply(lambda x: ' '.join(remove_stopWords(x['title'] + x['description'])), axis=1)
news['clean_text']  = news.apply(lambda x: x['text'].split(), axis = 1)
#get clean data
common_words_ct = Counter([j for i in news['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 100)
word_track = {i: ind for ind, i in enumerate(common_words)}
index_track = {ind: i for ind, i in enumerate(common_words)}
news['index_num'] = news.swifter.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

In [35]:
#change it to any location you save your embeddings
vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/agnews/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = news, common_words = common_words)


In [ ]:
seed_topic_list  = [['government', 'military', 'war'],
 ['basketball', 'football', 'athlete'],
 ['stock', 'market', 'industry'],
 ['computer', 'telescope', 'software']]
labels = [[word_track[j] for j in i] for i in seed_topic_list ]

In [84]:
class NSSTM(NTM):
    """
    A class used for negative sampling based semi-supervised topic modeling
    
    Attributes
    ----------
    beta
    gamma
    diversity_penalty: coefficients for diversity penalty
    index: a list of index of keywords
    sample
    
    Methods:
    --------
    forward(logit)
        a disctionary of loss function
    
    
    
    """
    def __init__(self, hidden, normal, h_to_z, topics, diversity_penalty, 
                 index, beta = 1, gamma = 1, iter1=10, iter2=20, sample = 20):
        # h_to_z will output probabilities over topics
        super(NSSTM, self).__init__(hidden, normal, h_to_z, topics)
        self.beta = beta
        self.gamma = gamma
        self.diversity_penalty = diversity_penalty
        self.index = index
        self.beta = beta
        self.gamma = gamma
        self.iter1 = iter1
        self.iter2 = iter2
        self.sample = sample
    def forward(self, x, n_sample=1, epoch = 0):
        stat = super(NSSTM, self).forward(x, n_sample)
        loss = stat['rec_loss'] 
        #penalty is mean - variance standard deviation
        if self.index == [] or epoch < self.iter1:
            self.ppenalty = 0
        else:
            self.ppenalty = negative_sampling_prior(self.topics.get_topics(),self.index, self.topics.embedding, epoch,
                                                   self.beta, self.gamma, self.iter2, self.sample)
        dpenalty, _, _ = topic_covariance_penalty(self.topics.topic_emb)
        stat.update({
            #loss add some penalty
            'loss': loss + self.ppenalty  + dpenalty * self.diversity_penalty,
            'penalty': self.ppenalty  + dpenalty * self.diversity_penalty,
            'prior': self.ppenalty
        })

        return stat
    

In [86]:
numb_embeddings = len(seed_topic_list) + 1
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1], 64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
embedding.weight.requires_grad=False
topics = EmbTopic(embedding = embedding,
                  k = numb_embeddings)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NSSTM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics,
            #prior_penalty = 1, 
            diversity_penalty = 0, 
            iter1 = 10,
            iter2 = 15,
            beta = 0.25,
            gamma = 0.25,
            index = labels
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=20)


epochs = 20
for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


162.1897666296725 12.498382521999766 0.0
155.44772659334294 32.57520895268617 0.0
148.6199530009776 73.15312489263539 0.0
144.61395224630198 105.49415720144569 0.0
143.23828777321367 124.69790193927822 0.0
142.76595254340913 146.85595823503508 0.0
142.5907206586175 173.02774246313425 0.0
142.50005804196095 198.17312342271623 0.0
142.43575299879126 221.75990366986565 0.0
142.3849003187883 248.94229230087703 0.0
142.7321109405713 269.71429638567764 tensor(5.1672, device='cuda:0', grad_fn=<DivBackward0>)
142.8392647293839 289.7628602310539 tensor(4.9211, device='cuda:0', grad_fn=<DivBackward0>)
142.8390905709663 306.4072947552972 tensor(4.8684, device='cuda:0', grad_fn=<DivBackward0>)
142.82803775811755 322.18561584863073 tensor(4.8394, device='cuda:0', grad_fn=<DivBackward0>)
142.8038824280696 334.1128047463228 tensor(4.8220, device='cuda:0', grad_fn=<DivBackward0>)
142.78704646909668 346.5145993100555 tensor(4.8078, device='cuda:0', grad_fn=<DivBackward0>)
142.82387674490272 354.9600032

In [87]:
gc.collect()
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:10] ] for i in range(numb_embeddings)]

#visualize keywords
mapping = [emb[:, i].mean(1).argmax() for i in labels]

data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
z = model.hidden(data_batch)
z, _ = model.normal(z)
z = model.h_to_z(z)
zz = torch.stack([z[:, i] for i in mapping]).T
zz = zz.cpu().detach().numpy()
# zz[:,0] = zz[:,0] * np.sqrt(0.219)
# zz[:,1] = zz[:,1]* np.sqrt(0.3826)
# zz[:,2] = zz[:,2]* np.sqrt(0.3106)
# zz[:,3] = zz[:,3] * np.sqrt(0.08724) 
y_pred = zz.argmax(1)
y_true = news['class'].iloc[indices].values - 1
accuracy = np.sum(y_pred == y_true)/news.shape[0]
coherence_score = get_topic_coherence(X.toarray(), seed_topic_list, word_track)
diversity_score = np.mean(diversity(topics))
macro = f1_score(y_true, y_pred, average='macro')
micro = f1_score(y_true, y_pred, average='micro')
print('The accuracy of the model is ' + str(accuracy))
print('The Diversity of the model is ' + str(diversity_score) )
print('The F1 macro score of the model is ' + str(macro))
enc = OneHotEncoder(handle_unknown='ignore')
y_real = enc.fit_transform(y_true.reshape(-1, 1)).toarray()
aucroc = roc_auc_score(y_real, zz, multi_class = 'ovo')      
print(accuracy, diversity_score, macro, aucroc)


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


The accuracy of the model is 0.8023
The Diversity of the model is 0.8400000000000001
The F1 macro score of the model is 0.798388240829509
0.8023 0.8400000000000001 0.798388240829509 0.9269292310185185


In [70]:
zz

array([[0.20235707, 0.19664623, 0.18870987, 0.18870987],
       [0.20235707, 0.19664623, 0.18870987, 0.18870987],
       [0.20235707, 0.19664623, 0.18870987, 0.18870987],
       ...,
       [0.20235707, 0.19664623, 0.18870987, 0.18870987],
       [0.20235707, 0.19664623, 0.18870987, 0.18870987],
       [0.20235707, 0.19664623, 0.18870987, 0.18870987]], dtype=float32)